# Gray - Scott Model of Reaction - Diffusion

A Reaction-Diffusion Model is a mathematical model which generally describes the interdependent interaction between objects in time and space. They are often used in chemistry to simulate the interaction between various chemical interactions in a given space. The Gray - Scott Model is one such reaction diffusion model.

The system in continuous form is described by the system of differential equations,
$$ \frac{\partial u}{\partial t} = D_u \nabla^2 u - uv^2 + f(1-u)$$
$$ \frac{\partial v}{\partial t} = D_v \nabla^2 v - uv^2 - (f+k)v$$

where $u, v$ are the associated substance spaces in euclidian space reacting with one another, $D_u$ is the diffusion coefficient of $u$, $D_v$ is the diffusion coefficient of $v$, $f$ is the feed rate of $u$, $k$ is the kill rate of $v$. $\nabla^2$ describes the laplacian of $u$ and $v$.

In the discretized form this looks like, 
$$ A' = A + (D_A \nabla^2 A - AB^2 + f(1-a))\Delta t $$
$$ B' = B + (D_B \nabla^2 B - AB^2 - (k+f)B)\Delta t $$

where $A$ and $B$ are the two substances in discretized space, $\Delta t$ is the discrete time step interval, and $\nabla^2$ describes the laplacian of $A$ and $B$. 

You can think of $\nabla^2$ being the $f$ in $f(x)$ but in this case, it would be $\nabla^2(x)$. Be careful here, because this isn't the gradient function $\nabla(x)$ squared, $\nabla^2$ is notation to describe the laplace operator, which is a function used to describe the divergence of the gradient of a scalar function in Euclidian space.




In [ ]:
# Packages Used: gray_scott.jl

## Initalization

In [ ]:
function init(n)

    u = ones((n+2, n+2));
    v = zeros((n+2, n+2));

    X_Mesh = (0:(n+1))' .* ones(n+2)/(n+1);
    Y_Mesh = (ones(n+2)/(n+1))' .* (0:(n+1));
    
    for i ∈ 1:n+2, j ∈ 1:n+2
        if X_Mesh[i,j] > 0.4 && X_Mesh[i,j] < 0.6 && Y_Mesh[i,j] > 0.4 && Y_Mesh[i,j] < 0.6
            u[i,j] = 0.5;
            v[i,j] = 0.25;
        end
    end

    return u, v;
end

## Laplacian 

In [ ]:
function laplacian(u,n)

    # Second Order Finite Difference
    laplacian = zeros(n,n);

    for i ∈ 2:n+1, j ∈ 2:n+1
        laplacian[i-1,j-1] = u[i-1,j] + u[i+1,j] + u[i,j-1] + u[i,j+1] - (4 * u[i,j])
    end

    return laplacian;
end

## Boundary Conditions

In [ ]:
function periodic_bc(u)
    # Periodic Boundary Conditions
    u[1,:] = u[end-1,:]
    u[end,:] = u[2,:]
    u[:,1] = u[:,end-1]
    u[:,end] = u[:,2]
end

## Gray-Scott Algorithm

In [ ]:
function gray_scott(U, V, Du, Dv, f, k)
    u, v = U[2:end-1, 2:end-1], V[2:end-1, 2:end-1];

    ∇²U = laplacian(U, n);
    ∇²V = laplacian(V, n);

    uv² = u .* v.^2;
    u += Du*∇²U .- uv² .+ f*(1 .- u);
    v += Dv*∇²V .+ uv² .- (f+k)*v;

    U[2:end-1,2:end-1] = u;
    V[2:end-1,2:end-1] = v;

    periodic_bc(U);
    periodic_bc(V);

    return U, V;
end

## Simulation and Visualization

In [ ]:
include("gray_scott.jl")

function main()

    #Constants:
    f = 0.0545;
    k = 0.6200;
    Du = .1;
    Dv = .05;
    n = 10;
    time_start = 1;
    time_end = 100;
    
    U, V = init(n);

    for time in time_start:time_end
        U, V = gray_scott(U, V, Du, Dv, f, k, n);
    end

end

main()